In [60]:
import callbacks

MASTER_ALL, MASTER_PID, DATE_MAPPER = callbacks.serve_data(ret=True,serve_local=True)

In [64]:
sub_df = MASTER_ALL.reset_index()


In [80]:
import random
pids = random.sample(range(0,3000),4)
last_date = sub_df['Date'].iloc[-1]


,pid,Date,country,province,state,county,granularity,Text_Confirmed,Text_Deaths,confirmed,...,deaths,deaths_lower,deaths_upper,forcast,lat,lon,CSize,DSize,CColor,DColor
0,0,2020-01-22,Afghanistan,N/A,N/A,N/A,country,Afghanistan<br> Total Cases: 0,Afghanistan<br> Total Deaths: 0,0.000000e+00,...,0.000000,0.000000,0.000000,False,33.000000,65.000000,0.0,0.000000,white,white
1,0,2020-01-23,Afghanistan,N/A,N/A,N/A,country,Afghanistan<br> Total Cases: 0,Afghanistan<br> Total Deaths: 0,0.000000e+00,...,0.000000,0.000000,0.000000,False,33.000000,65.000000,0.0,0.000000,white,white
2,0,2020-01-24,Afghanistan,N/A,N/A,N/A,country,Afghanistan<br> Total Cases: 0,Afghanistan<br> Total Deaths: 0,0.000000e+00,...,0.000000,0.000000,0.000000,False,33.000000,65.000000,0.0,0.000000,white,white
3,0,2020-01-25,Afghanistan,N/A,N/A,N/A,country,Afghanistan<br> Total Cases: 0,Afghanistan<br> Total Deaths: 0,0.000000e+00,...,0.000000,0.000000,0.000000,False,33.000000,65.000000,0.0,0.000000,white,white
4,0,2020-01-26,Afghanistan,N/A,N/A,N/A,country,Afghanistan<br> Total Cases: 0,Afghanistan<br> Total Deaths: 0,0.000000e+00,...,0.000000,0.000000,0.000000,False,33.000000,65.000000,0.0,0.000000,white,white
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272003,3090,2020-04-14,worldwide,NaN,NaN,NaN,NaN,Worldwide <br>,NaN,2.030070e+06,...,129352.879375,125799.841161,132905.917589,True,3730.397617,3207.531461,0.0,665.567689,white,#f01c19
272004,3090,2020-04-15,worldwide,NaN,NaN,NaN,NaN,Worldwide <br>,NaN,2.117517e+06,...,136482.654022,131452.905117,141512.402927,True,3730.397617,3207.531461,0.0,1500.000000,white,#ff0000
272005,3090,2020-04-16,worldwide,NaN,NaN,NaN,NaN,Worldwide <br>,NaN,2.206993e+06,...,143691.244846,137044.438658,150338.051034,True,3730.397617,3207.531461,0.0,1500.000000,white,#ff0000
272006,3090,2020-04-17,worldwide,NaN,NaN,NaN,NaN,Worldwide <br>,NaN,2.295216e+06,...,151155.750717,142663.336453,159648.164980,True,3730.397617,3207.531461,0.0,1500.000000,white,#ff0000


In [21]:
key_values=pd.DataFrame(list(key_values.values()),index=key_values.keys(),columns=['name'])

key_values[key_values['name'].str.strip().isin(['Worldwide','US'])].index

In [ ]:
worldwide_df = mater_df[mater_df['granularity']=='country'].groupby(['Date','forcast']).sum().reset_index()

In [ ]:
worldwide_df['country'] = 'worldwide'

In [ ]:
mater_df = mater_df.append(worldwide_df).reset_index(drop=True)

In [ ]:
latest_date = mater_df[mater_df['forcast'] == False].groupby('granularity').tail(1)['Date'].min()

In [ ]:
mater_df[mater_df['Date']==latest_date].reset_index(drop=True)

In [3]:
MASTER_ALL.to_pickle('Data/MASTER_ALL.pkl',compression='gzip')

In [4]:
MASTER_PID.to_pickle('Data/MASTER_PID.pkl',compression='gzip')

In [ ]:
from_pickle = pd.read_pickle('MASTER_ALL.pkl',compression='gzip')

In [ ]:
key_values = dict(zip(list(MASTER_PID.index),list(MASTER_PID['Text_Confirmed'].str.split('<br>').str.get(0))))
[{'label':key_values[x].replace('US','United States'), 'value':x} for x in key_values]

In [ ]:
from_pickle.reset_index()['Date']

In [ ]:
max_size = 1500

bins, ret_bins = pd.qcut(MASTER_ALL[MASTER_ALL['confirmed'] >= 1]['confirmed'], q=[
                             0, .5, 0.6, 0.70, 0.75, 0.8, 0.85, 0.9, 0.95, 0.999, 1], duplicates='drop', retbins=True)    
yellows = ["#606056", "#6e6e56", "#7b7c55", "#898a54", "#979953",
           "#a5a850", "#b4b74d", "#c3c649", "#d2d643", "#e2e53c"]
reds = ["#5a4f4f", "#704d4d", "#854b4a", "#994746", "#ac4340",
        "#be3c3a", "#cf3531", "#e02a27", "#f01c19", "#ff0000"]
labels = np.geomspace(1, max_size, num=len(ret_bins)-1)
MASTER_ALL['CSize'] = pd.cut(MASTER_ALL['confirmed'],bins=ret_bins,labels=labels).astype(float).fillna(0)
MASTER_ALL['DSize'] = pd.cut(MASTER_ALL['deaths'],bins=ret_bins,labels=labels).astype(float).fillna(0)
MASTER_ALL['CColor'] = pd.cut(MASTER_ALL['confirmed'], bins=ret_bins, labels=yellows).astype(str).replace({'nan': 'white'})
MASTER_ALL['DColor'] = pd.cut(MASTER_ALL['deaths'], bins=ret_bins, labels=reds).astype(str).replace({'nan': 'white'})

In [ ]:
from plotly import graph_objects as go
from datetime import timedelta
mapbox_style = 'mapbox://styles/jwillis0720/ck89nznm609pg1ipadkyrelvb'
mapbox_access_token = open('./.mapbox_token').readlines()[0]
def get_closest_inerval(n,s='max'):
    import math
    d = math.floor(math.log(abs(n)) / math.log(10)) 
    scale = 10**d
    if s == 'max':
        return math.ceil(n/scale) *scale
    return math.floor(n/scale) *scale

def plot_map(dataframe,metrics,zoom,center):
    fig = go.Figure()
    data_traces = []
    if not metrics or dataframe.empty:
        data_traces.append(go.Scattermapbox(
            lon=[],
            lat=[]
        ))

    dataframe = dataframe.reset_index()
    dataframe = dataframe[dataframe['Date'] == latest_date + timedelta(days=2)]
    dataframe = dataframe[dataframe['CSize'] > 0].sort_values('CSize')[::-1]
    if 'confirmed' in metrics and not dataframe.empty:
        # First Do Confirmed
        gb_confirmed = dataframe.groupby('CSize')
        
        ##Do this so we can sort them if need be
        for group in gb_confirmed:
            plotting_df = group[1]
            sizes = plotting_df['confirmed']
            ##This should be so, but okay
            sizes[sizes < 1] = 1
            name = "{}-{}".format(get_closest_inerval(int(plotting_df['confirmed'].min()),'min'),
                                  get_closest_inerval(int(plotting_df['confirmed'].max()),'max'))
            data = go.Scattermapbox(
                lon=plotting_df['lon'],
                lat=plotting_df['lat'],
                customdata=plotting_df['pid'],
                text=plotting_df['Text_Confirmed'],
                hoverinfo='text',
                name=name,
                mode='markers',
                marker=dict(
                    opacity=0.85,
                    sizemin=3,
                    size=plotting_df['CSize'],
                    color=plotting_df['CColor'],
                    sizemode='area'
                )
            )
            fig.add_trace(data)
            #data_traces.append(data)

    if 'deaths' in metrics and not dataframe.empty:
        #Second Do Confirmed
        dataframe = dataframe[dataframe['deaths'] > 1]
        gb_deaths = dataframe.groupby('DSize')
        
        ##Do this so we can sort them if need be
        for group in gb_deaths:
            plotting_df = group[1]
            ##This should be so, but okay
            print(int(plotting_df['deaths'].min()))
            name = "{}-{}".format(get_closest_inerval(int(plotting_df['deaths'].min()),'min'),
                                  get_closest_inerval(int(plotting_df['deaths'].max()),'max'))
            data = go.Scattermapbox(
                lon=plotting_df['lon'],
                lat=plotting_df['lat'],
                customdata=plotting_df['pid'],
                text=plotting_df['Text_Deaths'],
                hoverinfo='text',
                name=name,
                mode='markers',
                marker=dict(
                    opacity=0.85,
                    sizemin=3,
                    size=plotting_df['DSize'],
                    color=plotting_df['DColor'],
                    sizemode='area'
                )
            )
            fig.add_trace(data)

    annotations = []
    layout = dict(
        autosize=True,
        showlegend=True,
        mapbox=dict(
            accesstoken=mapbox_access_token,
            style=mapbox_style,
            zoom=zoom,
            center=center
        ),
        hovermode="closest",
        margin=dict(r=0, l=0, t=0, b=0),
        dragmode="pan",
        legend=dict(
            x=0.00,
            y=0.00,
            orientation='v',
            traceorder="normal",
            font=dict(
                family="Montserrat",
                color="white",
                size=16
            ),
            bgcolor='rgba(0,0,0,0.6)'
            #
        ),
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)'
    )
    fig.update_layout(layout)
    return fig
    #return {'data': data_traces, 'layout': layout}
gb = plot_map(MASTER_ALL,['confirmed','deaths'],None,None)
gb.show()

In [ ]:
10**d